In [1]:
import os
import config
import numpy as np
import pandas as pd
from scipy.signal import resample
from scipy.stats import mode

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


In [2]:
data_dir = "../../data/dataset/raw/"
subjects = os.listdir(data_dir)
gestures = config.GESTURES
learning_rate = 3e-4

In [3]:
dataset = pd.DataFrame()

for subject in subjects:
    for gesture in config.GESTURES:
        gesture_dir = os.path.join(data_dir, subject, gesture)
        recordings = os.listdir(gesture_dir)
        for recording in recordings:
            file_path = os.path.join(gesture_dir, recording)
            data = pd.read_csv(file_path)
            data.drop(columns=["time"], inplace=True)
            data = data.apply(resample, args=(config.SEGMENT_LEN, None, 0))

        #     # ... calculating distance of the index finger
        #     data["drf1"] = ((data["rf1x"] - data["rpx"]).pow(2) + \
        #             (data["rf1z"] - data["rpz"]).pow(2)).pow(0.5)

        #     data["dlf1"] = ((data["lf1x"] - data["lpx"]).pow(2) + \
        #             (data["lf1z"] - data["lpz"]).pow(2)).pow(0.5)

            data["drf0x"] = data["rf0x"] - data["rpx"]
            data["drf0y"] = data["rf0y"] - data["rpy"]
            data["drf0z"] = data["rf0z"] - data["rpz"]

            data["drf1x"] = data["rf1x"] - data["rpx"]
            data["drf1y"] = data["rf1y"] - data["rpy"]
            data["drf1z"] = data["rf1z"] - data["rpz"]

            data["label"] = config.GESTURES.index(gesture)

            dataset = pd.concat([dataset, data])

dataset.reset_index(inplace=True)
dataset

,index,rpx,rpy,rpz,lpx,lpy,lpz,rf0x,rf0y,rf0z,...,lf4x,lf4y,lf4z,drf0x,drf0y,drf0z,drf1x,drf1y,drf1z,label
0,0,8.653286,31.603047,13.038922,-19.162080,73.210373,-65.965233,8.945325,32.312522,5.471747,...,-24.855394,69.659697,-69.655715,0.292039,0.709474,-7.567175,-1.119610,-2.175523,-19.318795,0
1,1,46.783169,159.504625,48.771987,-86.885594,331.953882,-299.102633,-19.527602,153.864205,20.413757,...,-112.700485,315.854242,-315.836188,-66.310771,-5.640420,-28.358229,-31.626423,19.242958,-90.162293,0
2,2,45.729978,156.833091,49.532929,-86.360606,329.948118,-297.295367,-13.216102,152.150678,20.757329,...,-112.019515,313.945758,-313.927812,-58.946080,-4.682413,-28.775599,-28.722176,16.175299,-89.333336,0
3,3,46.460595,158.806056,49.202424,-86.885594,331.953882,-299.102633,-16.997866,153.548878,20.578893,...,-112.700485,315.854242,-315.836188,-63.458461,-5.257177,-28.623531,-30.529689,18.009856,-90.054869,0
4,4,45.868898,157.107912,49.332968,-86.360606,329.948118,-297.295367,-14.341596,152.285486,20.660206,...,-112.019515,313.945758,-313.927812,-60.210494,-4.822425,-28.672761,-29.234033,16.733200,-89.379020,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298747,251,73.231671,225.816270,79.739317,-6.578107,170.633864,-15.835375,-18.524249,277.239646,77.348978,...,-72.908035,139.249558,-0.144228,-91.755921,51.423376,-2.390339,-7.320679,43.227572,-99.068574,13
298748,252,72.056050,223.580205,79.528568,-6.539413,169.630136,-15.742225,-18.421219,275.171833,76.917692,...,-72.479165,138.430442,-0.143380,-90.477269,51.591628,-2.610876,-6.112769,43.640480,-98.547592,13
298749,253,72.647956,224.087382,79.751549,-6.578107,170.633864,-15.835375,-18.223585,276.072370,78.112803,...,-72.908035,139.249558,-0.144228,-90.871541,51.984988,-1.638746,-6.740097,45.782164,-97.780805,13
298750,254,73.700492,225.709217,82.509602,-6.539413,169.630136,-15.742225,-17.702200,269.856952,74.264817,...,-72.479165,138.430442,-0.143380,-91.402692,44.147735,-8.244785,-9.944349,41.872583,-99.561506,13


In [4]:
features = dataset[config.INFERENCE_FEATURES + ["label"]]
features

,rpx,rpy,rpz,rf0x,rf0y,rf0z,rf1x,rf1y,rf1z,label
0,8.653286,31.603047,13.038922,8.945325,32.312522,5.471747,7.533677,29.427524,-6.279873,0
1,46.783169,159.504625,48.771987,-19.527602,153.864205,20.413757,15.156746,178.747583,-41.390306,0
2,45.729978,156.833091,49.532929,-13.216102,152.150678,20.757329,17.007801,173.008390,-39.800407,0
3,46.460595,158.806056,49.202424,-16.997866,153.548878,20.578893,15.930907,176.815911,-40.852445,0
4,45.868898,157.107912,49.332968,-14.341596,152.285486,20.660206,16.634865,173.841112,-40.046052,0
...,...,...,...,...,...,...,...,...,...,...
298747,73.231671,225.816270,79.739317,-18.524249,277.239646,77.348978,65.910993,269.043843,-19.329257,13
298748,72.056050,223.580205,79.528568,-18.421219,275.171833,76.917692,65.943281,267.220685,-19.019024,13
298749,72.647956,224.087382,79.751549,-18.223585,276.072370,78.112803,65.907860,269.869545,-18.029256,13
298750,73.700492,225.709217,82.509602,-17.702200,269.856952,74.264817,63.756143,267.581800,-17.051904,13


In [5]:
import tensorflow as tf

from tensorflow.keras import layers, models, losses, optimizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [6]:
scaler = MinMaxScaler()
X = features.drop(columns=["label"]).to_numpy()
X = scaler.fit_transform(X)
X = X.reshape((-1, config.SEGMENT_LEN, len(config.INFERENCE_FEATURES)))
y = features["label"].to_numpy().reshape((-1, config.SEGMENT_LEN))
y, _ = mode(y, axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)


In [7]:
def conv_block_1d():
    inputs = layers.Input(shape=(config.SEGMENT_LEN, 1))
    x = layers.BatchNormalization()(inputs)
    x = layers.Conv1D(8, 3, activation="selu")(x)
    x = layers.Conv1D(8, 3, activation="selu")(x)
    x = layers.MaxPool1D(2)(x)
    x = layers.Conv1D(8, 3, activation="selu")(x)
    x = layers.Conv1D(8, 3, activation="selu")(x)
    x = layers.MaxPool1D(2)(x)
    x = layers.Conv1D(16, 3, activation="selu")(x)
    x = layers.Conv1D(16, 3, activation="selu")(x)
    x = layers.MaxPool1D(2)(x)
    x = layers.Conv1D(16, 3, activation="selu")(x)
    x = layers.Conv1D(16, 3, activation="selu")(x)
    x = layers.MaxPool1D(2)(x)
    x = layers.Flatten()(x)
    output = layers.Dense(64)(x)

    return inputs, output

def get_model(n_channels: int):
    inputs = []
    features = []

    for _ in range(n_channels):
        input_1d, features_1d = conv_block_1d()
        inputs.append(input_1d)
        features.append(features_1d)

    x = layers.concatenate(features, axis=-1)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.5)(x)
    output = layers.Dense(len(gestures), activation="softmax")(x)

    return models.Model(inputs, output)

In [8]:
model = get_model(n_channels=len(config.INFERENCE_FEATURES))

loss = losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer = optimizers.Adam(learning_rate=learning_rate)
model.compile(
    loss=loss,
    optimizer=optimizer,
    metrics=["accuracy"]
)

In [9]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=70,
        restore_best_weights=True
    )
]

history = model.fit(
    x=np.split(X_train, len(config.INFERENCE_FEATURES), axis=-1),
    y=y_train,
    validation_data=(
        np.split(X_test, len(config.INFERENCE_FEATURES), axis=-1),
        y_test
    ),
    batch_size=32,
    epochs=300,
    verbose=1,
    callbacks=callbacks
)

Epoch 1/300
25/25 [==============================] - 6s 82ms/step - loss: 3.2631 - accuracy: 0.0960 - val_loss: 2.6246 - val_accuracy: 0.0933
Epoch 2/300
25/25 [==============================] - 1s 39ms/step - loss: 2.3580 - accuracy: 0.2356 - val_loss: 2.5314 - val_accuracy: 0.0933
Epoch 3/300
25/25 [==============================] - 1s 39ms/step - loss: 2.0728 - accuracy: 0.3265 - val_loss: 2.3696 - val_accuracy: 0.2150
Epoch 4/300
25/25 [==============================] - 1s 40ms/step - loss: 1.8344 - accuracy: 0.4033 - val_loss: 2.1361 - val_accuracy: 0.4845
Epoch 5/300
25/25 [==============================] - 1s 39ms/step - loss: 1.5858 - accuracy: 0.4917 - val_loss: 1.8961 - val_accuracy: 0.5622
Epoch 6/300
25/25 [==============================] - 1s 40ms/step - loss: 1.3692 - accuracy: 0.5647 - val_loss: 1.5726 - val_accuracy: 0.6684
Epoch 7/300
25/25 [==============================] - 1s 41ms/step - loss: 1.2771 - accuracy: 0.5941 - val_loss: 1.3555 - val_accuracy: 0.7358
Epoch 

In [11]:
import matplotlib.pyplot as plt

plt.rcParams.update({
    # "text.usetex": True,
    "font.family": "serif",
    # "font.serif": ["Computer Modern Roman"],
    "font.size": 22,
    "text.color": "#212121",
    "axes.edgecolor": "#212121",
    "xtick.color": "#212121",
    "ytick.color": "#212121",
    "axes.labelcolor": "#212121",
    'legend.frameon': False,
})

fig = plt.figure(figsize=(8, 6))
ax = fig.gca()
ax.plot(history.history["loss"], "-", color="#212121", label="Train Loss")
ax.plot(history.history["val_loss"], "--", color="#212121", label="Validation Loss")
ax.set_xlabel("Epoch")
ax.set_ylabel("Loss")
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.title("Learning Curves")
plt.legend()
plt.tight_layout()
plt.show()

RuntimeError: Failed to process string with tex because latex could not be found

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 576x432 with 1 Axes>

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
import joblib
joblib.dump(scaler, "../../models/scaler.joblib")
model.save("../../models/stack_cnn")